## Using ML.NET in F# with Polyglot Notebooks
By Matt Eland

### Importing ML.NET

In [49]:
// Install the Microsoft.ML packages we're going to use for ML.NET
#r "nuget:Microsoft.ML"

Installed Packages Microsoft.ML, 2.0.0

In [50]:
// Create an ML Context. We'll use this for all ML operations
let context = new MLContext();

### Loading Data
Source: [London Weather Data](https://www.kaggle.com/datasets/emmanuelfwerr/london-weather-data)

In [51]:
type WeatherInput =
    {
        // Ignore date column
        [<LoadColumn(1)>]
        CloudCover: Single // in oktas
        [<LoadColumn(2)>]
        Sunshine: Single // in hours
        [<LoadColumn(3)>]
        GlobalRads: Single // Global radiation in Watt / square meter
        [<LoadColumn(4)>]
        MaxTemp: Single // Celsius
        [<LoadColumn(5)>]
        MeanTemp: Single // Celsius
        [<LoadColumn(6)>]
        MinTemp: Single // Celsius
        [<LoadColumn(7)>]
        Precipitation: Single // This is the precipitation in millimeters. This is what we want to predict
        [<LoadColumn(8)>]
        Pressure: Single // In Pascals
        // Ignore snow depth since that won't be known at time of prediction and is related to Precipitation
    }

In [52]:
// Load the data
let data = context.Data.LoadFromTextFile<WeatherInput>("london_weather.csv", hasHeader = true, separatorChar=',')

In [53]:
// Split into train and test splits to detect overfitting
let split = context.Data.TrainTestSplit(data, testFraction = 0.2)

let trainSet = split.TrainSet
let testSet = split.TestSet

In [54]:
let processPipeline = EstimatorChain()
                        .Append(context.Transforms.CopyColumns("Label", "Precipitation"))
                        .Append(context.Transforms.NormalizeMeanVariance("CloudCover", "CloudCover"))
                        .Append(context.Transforms.NormalizeMeanVariance("Sunshine", "Sunshine"))
                        .Append(context.Transforms.NormalizeMeanVariance("GlobalRads", "GlobalRads"))
                        .Append(context.Transforms.NormalizeMeanVariance("MaxTemp", "MaxTemp"))
                        .Append(context.Transforms.NormalizeMeanVariance("MeanTemp", "MeanTemp"))
                        .Append(context.Transforms.NormalizeMeanVariance("MinTemp", "MinTemp"))
                        .Append(context.Transforms.NormalizeMeanVariance("Pressure", "Pressure"))
                        .Append(context.Transforms.Concatenate("Features", [|"CloudCover";"Sunshine";"GlobalRads";"MaxTemp";"MeanTemp";"MinTemp";"Pressure"|]))
                        .AppendCacheCheckpoint(context)

In [55]:
// Select the algorithm we want
let trainer = context.Regression.Trainers.OnlineGradientDescent(labelColumnName = "Label", featureColumnName = "Features")

let pipeline = processPipeline.Append(trainer)

In [56]:
// Train the model
let model = pipeline.Fit(trainSet)

model

index type FeatureColumnName FeatureColumnType Model 0 Microsoft.ML.Transforms.ColumnCopyingTransformer 1 Microsoft.ML.Transforms.NormalizingTransformer 2 Microsoft.ML.Transforms.NormalizingTransformer 3 Microsoft.ML.Transforms.NormalizingTransformer 4 Microsoft.ML.Transforms.NormalizingTransformer 5 Microsoft.ML.Transforms.NormalizingTransformer 6 Microsoft.ML.Transforms.NormalizingTransformer 7 Microsoft.ML.Transforms.NormalizingTransformer 8 Microsoft.ML.Data.ColumnConcatenatingTransformer 9 Microsoft.ML.Data.RegressionPredictionTransformer<Microsoft.ML.Trainers.LinearRegressionModelParameters> Features Dimensions IsKnownSize ItemType Size RawType [ 7 ] True NumberDataViewType
 RawType: System.Single 7 Microsoft.ML.Data.VBuffer<System.Single> Weights Bias [ 1.1249702, -0.9436583, 0.09858767, -0.41163045, 0.29090133, 0.42288145, 0.36714587 ] 0.70410115

In [57]:
let testResults = model.Transform(testSet)

let testMetrics = context.Regression.Evaluate(testResults, "Label", "Score")

testMetrics

MeanAbsoluteError,MeanSquaredError,RootMeanSquaredError,LossFunction,RSquared
2.117368611215726,13.202526174072856,3.633528061550214,13.202526238818384,0.06655389348867957


In [58]:
context.Model.Save(model, data.Schema, "Model.zip")

In [72]:
let loadedModel, loadedSchema = context.Model.Load("Model.zip")

index type FeatureColumnName FeatureColumnType Model 0 Microsoft.ML.Transforms.ColumnCopyingTransformer 1 Microsoft.ML.Transforms.NormalizingTransformer 2 Microsoft.ML.Transforms.NormalizingTransformer 3 Microsoft.ML.Transforms.NormalizingTransformer 4 Microsoft.ML.Transforms.NormalizingTransformer 5 Microsoft.ML.Transforms.NormalizingTransformer 6 Microsoft.ML.Transforms.NormalizingTransformer 7 Microsoft.ML.Transforms.NormalizingTransformer 8 Microsoft.ML.Data.ColumnConcatenatingTransformer 9 Microsoft.ML.Data.RegressionPredictionTransformer<Microsoft.ML.Trainers.LinearRegressionModelParameters> Features Dimensions IsKnownSize ItemType Size RawType [ 7 ] True NumberDataViewType
 RawType: System.Single 7 Microsoft.ML.Data.VBuffer<System.Single> Weights Bias [ 1.1249702, -0.9436583, 0.09858767, -0.41163045, 0.29090133, 0.42288145, 0.36714587 ] 0.70410115

In [59]:
[<CLIMutable>]
type WeatherPrediction = {
    [<ColumnName("Score")>] 
    Precipitation: Single
}

let predictionEngine = context.Model.CreatePredictionEngine<WeatherInput, WeatherPrediction>(model)

In [60]:
let conditions = {
    CloudCover = 4f; 
    Sunshine=5.8f; 
    GlobalRads=50f; 
    MaxTemp=5.2f; 
    MinTemp=1.6f; 
    MeanTemp=3.7f; 
    Pressure=101170f;
    Precipitation=0f; // Not used, but part of input scheme
}

let prediction = predictionEngine.Predict(conditions)

prediction

Precipitation
1.0089107
